In [1]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [2]:
import os
import pandas as pd
import numpy as np
import joblib  
import json

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import (
    CLEANED_DATA_DIR,
    TEST_PREDICTIONS_DIR,
)
import project_utils.ml_utils as ml_utils
import project_utils.custom_transformers as custom_transformers

# Data pre-processing
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Model and random-search and cross-validation
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import (
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    ExtraTreesRegressor,
)

2025-12-16 22:48:56.254 | INFO     | project_utils.paths:<module>:19 - PROJ_ROOT path is: /Users/alexandre/Documents/GitHub/price_predictor


Note: All models in this study were trained without explicit outlier removal. This decision was based on empirical evaluation, which indicated that retaining outliers led to more robust models and improved generalization performance, whereas outlier removal consistently reduced validation stability and predictive accuracy.

# Tree Based Ensemble Methods (TBEM)

Considering the potential non-linearity of the problem, this notebook presents a comparative analysis of two ensemble regression tree methods: Gradient Boosting and Extremely Randomized Trees. Each algorithm is examined in detail in its respective section, with the aim of evaluating their predictive performance and suitability for the task.

In [3]:
df_train = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"), index_col="carID"
)
X_test = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"), index_col="carID"
)

X = df_train.drop("price", axis=1).copy()
y = df_train["price"].copy()

## Gradient Boosting

Gradient boosting is a powerful ensemble method that builds a model sequentially, where each new predictor is trained to correct the errors made by the previous one [1]. Although this process is often described as “fitting the residuals,” the algorithm more generally uses the negative gradient of the loss function with respect to the current predictions. This gradient represents the direction of steepest descent in function space, and each successive model is trained to approximate it. The ensemble is then updated by adding this new model to the current prediction, gradually improving performance through iterative functional gradient descent [2]. In our project, we use Gradient Boosted Regression Trees (GBRT), an ensemble technique in which successive shallow regression trees are added sequentially, each one trained to approximate the negative gradient of the loss function and correct the residual errors of the previous model.

In [4]:
# --- Preprocessing pipeline --- #
preprocessing_pipeline = joblib.load(
    os.path.join(PROJ_ROOT, "src", "project_utils", "full_preprocessor.joblib")
)

# --- GBR with log-transform target --- #
log_gbr_regressor = TransformedTargetRegressor(
    regressor=GradientBoostingRegressor(
        loss="absolute_error",
        learning_rate=0.1,
        n_estimators=300,
        subsample=0.8,
        max_depth=4,
        min_samples_leaf=5,
        n_iter_no_change=5,
        random_state=0,
    ),
    func=np.log1p,
    inverse_func=np.expm1,
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessing_pipeline),
        ("regressor", log_gbr_regressor),
    ]
)

# --- K-Fold CV ---#
cv = KFold(n_splits=5, shuffle=True, random_state=0)

scores = cross_val_score(
    model, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1)


# Fit random_search
model.fit(X, y)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('drop_filtered_features', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('group_imputer', ...), ('column_tra

In [5]:
# Export the full preprocessing pipeline (pre_pipeline + feature selection)
joblib.dump(preprocessing_pipeline, os.path.join(PROJ_ROOT, "src", "project_utils", "full_preprocessor.joblib"))

['/Users/alexandre/Documents/GitHub/price_predictor/src/project_utils/full_preprocessor.joblib']

In [6]:
# Compute evaluation metrics
model.fit(X, y)
y_pred = model.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

Training MAE: 1724.53
CV MAE mean: 1735.34
CV MAE std : 33.7


This model demonstrates markedly stronger and more well-balanced predictive performance compared to the linear regression baseline. The close alignment between the training error and the cross-validation (CV) error indicates that the model generalises well to unseen data and is not overfitting the training set.

In [7]:
# # Export the pipeline 
# joblib.dump(model, os.path.join(PROJ_ROOT, "src", "models", "GBR_model.joblib"))

In [8]:
# -- Export X_test predictions --#
pred_y_test = model.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "gbr_test_predictions.csv")
)

### Histogram-based Gradient Boosting Regression Tree (HGBRT)

This model operates similarly to Gradient Boosted Regression Trees (GBRT), with the key distinction that it discretises continuous variables into a fixed number of bins. This histogram-based representation greatly accelerates the tree-building process and, as a result, enhances computational efficiency without sacrificing predictive performance. The approach is known to be substantially faster than classical GBRT, particularly for datasets exceeding 10,000 observations [3]. Given that our training set contains approximately 76,000 samples, this efficiency gain provided a strong motivation to adopt the histogram-based method. An additional advantage is that the reduced computational cost enabled us to increase the number of iterations in the randomised hyperparameter search, thereby allowing a more extensive exploration of the parameter space and improving the likelihood of identifying an optimal model configuration.

Throughout this project, this model consistently demonstrated the strongest generalization performance. Consequently, an extensive random search was conducted to further optimize its hyperparameters. For transparency and reproducibility, the hyperparameter search space employed in this process was preserved and is available in the file hgbr_param_grid.py for reference. The optimal hyperparameter configuration identified during the search was exported to a JSON file and subsequently reloaded for use in the final HistGradientBoostingRegressor model.

In [9]:
# Import JSON file with best_params selected from the RandomSearch
RANDOM_SEARCH = os.path.join(PROJ_ROOT, "src", "models", "random_search")
with open(os.path.join(RANDOM_SEARCH, "best_hgbr_params.json"), "r") as f:
    best_params = json.load(f)

clean_params = {
    k.replace("regressor__regressor__", ""): v for k, v in best_params.items()
}

In [10]:
# --- HGBR with log-transform target --- #
log_tree_regressor = TransformedTargetRegressor(
    regressor=HistGradientBoostingRegressor(**clean_params),
    func=np.log1p,
    inverse_func=np.expm1,
)

model = Pipeline(
    [
        ("preprocessor", preprocessing_pipeline),
        ("regressor", log_tree_regressor),
    ]
)
# --- K-Fold CV ---#
cv = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(
    model, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1
)

In [11]:
# Compute evaluation metrics
model.fit(X, y)
y_pred = model.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

Training MAE: 1226.81
CV MAE mean: 1420.73
CV MAE std : 16.43


The HGBRT model demonstrates superior predictive performance relative to the GBRT. The close alignment between the training and CV errors indicates that the model generalises effectively and is not overfitting the training data. In addition, the reduction in the standard deviation of the CV scores suggests improved stability, with the model producing more robust and consistent predictions across different validation folds.


In [12]:
# Export the pipeline with the best parameters found via RandomizedSearchCV
joblib.dump(
    model, os.path.join(PROJ_ROOT, "src", "models", "HGBR_model.joblib")
)

['/Users/alexandre/Documents/GitHub/price_predictor/src/models/HGBR_model.joblib']

In [13]:
# -- Export X_test predictions --#
pred_y_test = model.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "hgbr_test_predictions.csv")
)

## Extremely randomized trees (ETR)

Like Random Forests, ETRs can rely on bootstrapped samples to introduce diversity into the training data, but they differ fundamentally in their split selection mechanism: whereas Random Forests search for the optimal split among a random subset of features, ETRs select both features and split thresholds entirely at random [4]. This additional layer of randomness increases model diversity and reduces variance, while the use of random thresholds substantially lowers the computational cost of tree construction, making Extra Trees more efficient than Random Forests, particularly when applied to large datasets.

In [14]:
# --- ETR with log-transform target --- #
extr = ExtraTreesRegressor(
    criterion="squared_error",
    random_state=0,
    bootstrap=True,
    n_jobs=-1,
)

log_extr = TransformedTargetRegressor(
    regressor=extr,
    func=np.log1p,
    inverse_func=np.expm1,
)

pipeline = Pipeline(
    [
        ("preprocessor", preprocessing_pipeline),
        ("regressor", log_extr),
    ]
)

# --- K-Fold CV ---#
cv = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(
    pipeline, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1
)
pipeline.fit(X, y)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('drop_filtered_features', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('group_imputer', ...), ('column_tra

In [15]:
# Compute evaluation metrics
y_pred = pipeline.predict(X)
mae = ml_utils.linear_evaluation_metrics(y, y_pred, verbose=False)[1]
print("Training MAE:", round(mae, 2))
mae_scores = -scores
print("CV MAE mean:", round(mae_scores.mean(), 2))
print("CV MAE std :", round(mae_scores.std(), 2))

Training MAE: 1008.01
CV MAE mean: 1430.51
CV MAE std : 26.37


In [16]:
# -- Export X_test predictions --#
pred_y_test = pipeline.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "hgbr_test_predictions.csv")
)


In [17]:
joblib.dump(pipeline, os.path.join(PROJ_ROOT, "src", "models", "ETR_model.joblib"))

['/Users/alexandre/Documents/GitHub/price_predictor/src/models/ETR_model.joblib']

We observe a reduction in the training error, while the CV error remains essentially unchanged. However, this discrepancy is not sufficient to conclude that the model is overfitting.

## References

[1] A. Géron, “Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow”.

[2] J. H. Friedman, “Greedy function approximation: A gradient boosting machine.,” The Annals of Statistics, vol. 29, no. 5, pp. 1189–1232, Oct. 2001, doi: 10.1214/aos/1013203451.

[3] Scikit-learn developers, “Histogram Gradient Boosting Regression Example,” scikit-learn: Machine Learning in Python, 2024. [Online]. Available: https://scikit-learn.org/stable/auto_examples/ensemble/plot_hgbt_regression.html.

[4] P. Geurts, D. Ernst, and L. Wehenkel, “Extremely randomized trees,” Mach Learn, vol. 63, no. 1, pp. 3–42, Apr. 2006, doi: 10.1007/s10994-006-6226-1.